In [1]:
from dotenv import load_dotenv

load_dotenv()

True

In [2]:
from langgraph.graph import StateGraph, START, END
from typing_extensions import TypedDict


class InputState(TypedDict):
    question: str


class OutputState(TypedDict):
    answer: str


class OverallState(InputState, OutputState):
    pass


def answer_node(state: InputState):
    return {"answer": "bye", "question": state["question"]}



builder = StateGraph(OverallState, input=InputState, output=OutputState)
builder.add_node(answer_node)
builder.add_edge(START, "answer_node")
builder.add_edge("answer_node", END)
graph = builder.compile()

print(graph.invoke({"question": "hi"}))

{'answer': 'bye'}


### Configurable Agent

In [3]:
from langchain_openai import ChatOpenAI
from langgraph.graph import END, START, StateGraph

model = ChatOpenAI()

In [4]:
from typing import TypedDict

class ChatMessages(TypedDict):
    question: str
    answer: str
    llm_calls: int

In [5]:
def call_model(state: ChatMessages):
    question = state['question']
    llm_calls = state.get('llm_calls', 0) # workaround
    state["llm_calls"] = llm_calls + 1
    print("LLM_CALLS:", llm_calls)
    response = model.invoke(input=question)
    state["answer"] = response.content
    return state

In [6]:
workflow = StateGraph(ChatMessages)

workflow.add_edge(START, "agent")
workflow.add_node("agent", call_model)
workflow.add_edge("agent", END)

graph = workflow.compile()

In [7]:
graph.invoke(input={"question": "Whats the highest mountain in the world?"})

LLM_CALLS: 0


{'question': 'Whats the highest mountain in the world?',
 'answer': 'Mount Everest',
 'llm_calls': 1}

### Input & Output 

In [8]:
from langgraph.graph import StateGraph, START, END
from typing_extensions import TypedDict


class InputState(TypedDict):
    question: str


class PrivateState(TypedDict):
    llm_calls: int


class OutputState(TypedDict):
    answer: str


class OverallState(InputState, PrivateState, OutputState):
    pass

In [9]:
workflow = StateGraph(OverallState, input=InputState, output=OutputState)

workflow.add_edge(START, "agent")
workflow.add_node("agent", call_model)
workflow.add_edge("agent", END)

graph = workflow.compile()

In [10]:
graph.invoke({"question": "Whats the highest mountain in the world?"})

LLM_CALLS: 0


{'answer': 'Mount Everest'}

### Add runtime configuration

In [11]:
from langgraph.graph import StateGraph, START, END
from typing_extensions import TypedDict
from langchain_core.runnables.config import RunnableConfig
from langchain.schema import SystemMessage, HumanMessage


def call_model(state: OverallState, config: RunnableConfig):
    language = config["configurable"].get("language", "English")
    system_message_content = "Respond in {}".format(language)
    system_message = SystemMessage(content=system_message_content)
    messages = [system_message, HumanMessage(content=state["question"])]
    response = model.invoke(messages)
    return {"answer": response}


In [12]:
workflow = StateGraph(ChatMessages)

workflow.add_edge(START, "agent")
workflow.add_node("agent", call_model)
workflow.add_edge("agent", END)

graph = workflow.compile()

In [13]:
config = {"configurable": {"language": "Spanish"}}
graph.invoke({"question": "What's the highest mountain in the world?"}, config=config)

{'question': "What's the highest mountain in the world?",
 'answer': AIMessage(content='La montaña más alta del mundo es el Monte Everest, con una altitud de 8,848 metros sobre el nivel del mar. Se encuentra en la cordillera del Himalaya, en la frontera entre Nepal y China.', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 48, 'prompt_tokens': 23, 'total_tokens': 71, 'completion_tokens_details': {'audio_tokens': None, 'reasoning_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': None, 'cached_tokens': 0}}, 'model_name': 'gpt-3.5-turbo-0125', 'system_fingerprint': None, 'finish_reason': 'stop', 'logprobs': None}, id='run-2d42a983-eead-4d0b-8e06-8c53f6312bc2-0', usage_metadata={'input_tokens': 23, 'output_tokens': 48, 'total_tokens': 71, 'input_token_details': {'cache_read': 0}, 'output_token_details': {'reasoning': 0}})}

https://langchain-ai.github.io/langgraph/how-tos/configuration/#configure-the-graph